# 0.Introduction
Use BERT to get the average vector of any word from multiple given sentences

# 1.Settings

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 1.1MB 47.6MB/s 
     |████████████████████████████████| 890kB 50.4MB/s 
     |████████████████████████████████| 2.9MB 43.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fe4bcf4c2b02af135e18ec492d8fae0d986e170189d34be5ff790e1d461e0d74
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
from transformers import BertModel, BertTokenizer

# 2.Load Models

In [3]:
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 3.Obtain word vectors from each sentence vector

In [4]:
s1 = "i eat an apple today ."
s2 = "you eat an apple today ."

In [5]:
inputs1 = tokenizer(s1, return_tensors="pt")
inputs2 = tokenizer(s2, return_tensors="pt")

In [6]:
inputs1

{'input_ids': tensor([[ 101, 1045, 4521, 2019, 6207, 2651, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
inputs2

{'input_ids': tensor([[ 101, 2017, 4521, 2019, 6207, 2651, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
with torch.no_grad():
  outputs1 = model(**inputs1)
  outputs2 = model(**inputs2)

In [9]:
last_hidden_state1 = outputs1.last_hidden_state[0]
last_hidden_state2 = outputs1.last_hidden_state[0]

In [10]:
last_hidden_state1

tensor([[ 0.0465,  0.1496, -0.3968,  ...,  0.0412,  0.5392,  0.5326],
        [ 0.2066,  0.0756, -0.3788,  ..., -0.0933,  1.1465,  0.3369],
        [-0.0625,  0.7618,  0.1640,  ...,  0.3169,  0.8769,  0.1139],
        ...,
        [-0.4008, -0.0939, -0.3059,  ..., -0.2158,  0.2630, -0.7132],
        [ 0.3049, -0.0819, -0.3699,  ...,  0.3447,  0.2585, -0.5883],
        [ 0.7464,  0.2905, -0.1306,  ...,  0.3689, -0.2140, -0.4930]])

In [11]:
apple1 = last_hidden_state1[4]
apple2 = last_hidden_state2[4]

In [12]:
cos = torch.nn.CosineSimilarity(dim=0)

In [13]:
cos(apple1, apple2)

tensor(1.)

In [14]:
torch.dot(apple1, apple2)/(torch.norm(apple1)*torch.norm(apple2) + 1e-8)

tensor(1.0000)

## 3.1 What is '**'?

In [15]:
def print_kwargs(**kwargs):
    print("Keyword arguments:", kwargs)

In [16]:
print_kwargs()

print_kwargs(num=1, text="aaa")

Keyword arguments: {}
Keyword arguments: {'num': 1, 'text': 'aaa'}


In [17]:
print_kwargs(**inputs1)

Keyword arguments: {'input_ids': tensor([[ 101, 1045, 4521, 2019, 6207, 2651, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


# 4.Main: single sentence
Create average target word vectors from a corpus.  
target words: 'apple', 'you'
1. convert word->id by tokenizer
1. get the ***word_id*** contained in ***target_words*** and the ***position*** in the sentence in the word_id
1. obtain sentence vectors by BERT
1. store the target word vectors

In [18]:
corpus = ["i eat an apple today .",
                "you eat an apple today .",
                "i drink an apple juice .",
                "you like an apple .",
                "an apple tree is heavy with fruit .",
                "this apple is sweet .",
                "you make a sweet apple pie .",
                "apple trees fructify .",
                "i buy an apple computer at an apple store .",
                "you buy an apple computer today ."]

In [19]:
target_words = ["apple", "you"]

## 4.1 Prepare word2id and id2word dictionaries from BERT vocaburary

In [20]:
 !wget https://raw.githubusercontent.com/microsoft/SDNet/master/bert_vocab_files/bert-base-uncased-vocab.txt

--2020-10-22 16:39:09--  https://raw.githubusercontent.com/microsoft/SDNet/master/bert_vocab_files/bert-base-uncased-vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘bert-base-uncased-vocab.txt’

bert-base-uncased-v 100%[===================>] 226.08K  --.-KB/s    in 0.007s  

2020-10-22 16:39:09 (30.1 MB/s) - ‘bert-base-uncased-vocab.txt’ saved [231508/231508]



In [21]:
word2id = {}
id2word = {}

In [22]:
id = 0
with open('bert-base-uncased-vocab.txt') as fp:
  for line in fp:
    word = line.strip()
    word2id[word] = id
    id2word[id] = word
    id += 1

In [23]:
word2id['apple']

6207

In [24]:
word2id['you']

2017

## 4.3 Store the target word vectors

In [25]:
target_ids = [word2id[word] for word in target_words if word in word2id]

In [26]:
print(f'{len(target_ids)} / {len(target_words)}')

2 / 2


In [34]:
target_vecs = torch.zeros([len(target_ids), 768])

In [29]:
targetid_freq = [0]*len(target_ids)

In [35]:
with torch.no_grad():
  for sentence in corpus:
    sentence_tokenized = tokenizer(sentence, return_tensors="pt")
    last_hidden_states = model(**sentence_tokenized).last_hidden_state[0]
    for i, input_id in enumerate(sentence_tokenized['input_ids'][0]):
      target_id = int(input_id)
      if target_id in target_ids:
        target_vecs[target_ids.index(target_id)] += last_hidden_states[i]
        targetid_freq[target_ids.index(target_id)] += 1

In [37]:
for target_id in target_ids:
  print(f'word: {id2word[target_id]}')
  print(f'vec: {target_vecs[target_ids.index(target_id)]}')

word: apple
vec: tensor([ 1.8931e+00,  5.8149e+00, -1.5698e+00, -3.0820e+00,  1.4313e+01,
         9.1385e+00, -3.2755e+00,  1.0480e+01, -4.8699e-01, -3.0038e+00,
        -2.6320e+00, -5.9830e-01, -2.1299e+00,  7.7760e-02, -3.0052e-02,
         3.0557e+00,  5.8750e+00,  9.7794e+00, -4.1202e+00,  3.2106e+00,
         3.1494e+00, -5.1906e+00, -1.2630e-01,  2.7443e+00,  1.0151e+01,
        -6.2851e+00, -4.6552e+00, -1.6419e+00, -1.5092e+00, -7.9071e+00,
        -4.2983e+00,  1.0407e+01,  1.1130e+01, -1.8547e+00, -7.3365e+00,
        -4.8821e+00, -7.3216e+00,  9.8755e-01, -8.9279e+00,  2.5594e+00,
        -3.9658e+00, -4.6125e+00,  5.0215e+00, -1.8313e+00,  3.0903e+00,
        -5.3100e+00, -8.5286e+00, -3.5770e+00,  8.6085e-01,  1.3234e+00,
        -3.7144e+00,  6.0103e+00, -3.0013e+00, -4.1651e+00,  3.6610e+00,
         1.0166e+01, -6.3988e-01, -1.0324e+01,  5.3186e+00,  2.3172e+00,
        -2.1985e+00, -3.9365e+00,  2.6749e+00, -4.2643e+00, -5.3746e+00,
         3.9527e+00,  3.4801e+00, 

## 4.4 Calcurate average target word vectors

In [39]:
target_vecs = {
    id2word[target_ids[i]]: target_vecs[i] / targetid_freq[i] for i in range(len(target_ids))
}

In [40]:
target_vecs

{'apple': tensor([ 1.7210e-01,  5.2862e-01, -1.4271e-01, -2.8018e-01,  1.3012e+00,
          8.3078e-01, -2.9777e-01,  9.5272e-01, -4.4272e-02, -2.7307e-01,
         -2.3927e-01, -5.4391e-02, -1.9362e-01,  7.0691e-03, -2.7320e-03,
          2.7779e-01,  5.3409e-01,  8.8904e-01, -3.7456e-01,  2.9187e-01,
          2.8631e-01, -4.7187e-01, -1.1482e-02,  2.4948e-01,  9.2279e-01,
         -5.7138e-01, -4.2320e-01, -1.4926e-01, -1.3720e-01, -7.1882e-01,
         -3.9075e-01,  9.4611e-01,  1.0118e+00, -1.6861e-01, -6.6695e-01,
         -4.4382e-01, -6.6560e-01,  8.9777e-02, -8.1163e-01,  2.3268e-01,
         -3.6053e-01, -4.1931e-01,  4.5650e-01, -1.6648e-01,  2.8094e-01,
         -4.8273e-01, -7.7533e-01, -3.2518e-01,  7.8259e-02,  1.2031e-01,
         -3.3767e-01,  5.4639e-01, -2.7284e-01, -3.7865e-01,  3.3282e-01,
          9.2420e-01, -5.8171e-02, -9.3855e-01,  4.8351e-01,  2.1065e-01,
         -1.9986e-01, -3.5786e-01,  2.4317e-01, -3.8767e-01, -4.8860e-01,
          3.5934e-01,  3.1637

# 5.Main: multi sentences

In [41]:
corpus = ["i eat an apple today .",
          "you eat an apple today .",
          "i drink an apple juice .",
          "you like an apple .",
          "an apple tree is heavy with fruit .",
          "this apple is sweet .",
          "you make a sweet apple pie .",
          "apple trees fructify .",
          "i buy an apple computer at an apple store .",
          "you buy an apple computer today ."]

In [42]:
target_words = ["apple", "you"]

In [43]:
target_ids = [word2id[word] for word in target_words if word in word2id]
print(target_ids)
set_target_ids = set(target_ids)

[6207, 2017]


In [44]:
def make_batch(corpus, batch_size):
  """ 
  :param corpus: list of sentences
  :param batch_size: batch_size
  :return: samples
  """ 
  sorted_sentence = sorted(corpus, key=lambda x: len(x.split()))
  samples = [sorted_sentence[i:i + batch_size] if i+batch_size<len(sorted_sentence) else sorted_sentence[i:]\
                   for i in range(0, len(sorted_sentence), batch_size)]
  return samples

In [45]:
from tqdm import tqdm

In [46]:
samples = make_batch(corpus, batch_size=3)
print(samples)

[['apple trees fructify .', 'you like an apple .', 'this apple is sweet .'], ['i eat an apple today .', 'you eat an apple today .', 'i drink an apple juice .'], ['you make a sweet apple pie .', 'you buy an apple computer today .', 'an apple tree is heavy with fruit .'], ['i buy an apple computer at an apple store .']]


In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_vecs = torch.zeros((len(target_ids), 768)).to(device)
print(f'target_vecs: {target_vecs.size()}')

targetid_freq = [0]*len(target_ids)

model.to(device)
with torch.no_grad():
  for sample in tqdm(samples):
    inputs = tokenizer(sample, return_tensors='pt', padding=True, truncation=True).to(device)
    outputs = model(**inputs).last_hidden_state
    input_ids = inputs['input_ids']
    for sent_id in range(outputs.shape[0]):
      for word_id in range(outputs.shape[1]):
        target_id = int(input_ids[sent_id][word_id])
        if target_id in set_target_ids:
          target_vecs[target_ids.index(target_id)] += outputs[sent_id][word_id]
          targetid_freq[target_ids.index(target_id)] += 1

  0%|          | 0/4 [00:00<?, ?it/s]

target_vecs: torch.Size([2, 768])


100%|██████████| 4/4 [00:00<00:00, 42.41it/s]


In [48]:
target_vecs = {
    id2word[target_ids[i]]: target_vecs[i] / targetid_freq[i] for i in range(len(target_ids))
}

In [49]:
target_vecs

{'apple': tensor([ 1.7210e-01,  5.2862e-01, -1.4271e-01, -2.8018e-01,  1.3012e+00,
          8.3078e-01, -2.9777e-01,  9.5272e-01, -4.4272e-02, -2.7307e-01,
         -2.3927e-01, -5.4391e-02, -1.9362e-01,  7.0693e-03, -2.7322e-03,
          2.7779e-01,  5.3409e-01,  8.8904e-01, -3.7456e-01,  2.9187e-01,
          2.8631e-01, -4.7187e-01, -1.1482e-02,  2.4948e-01,  9.2279e-01,
         -5.7138e-01, -4.2320e-01, -1.4926e-01, -1.3720e-01, -7.1882e-01,
         -3.9075e-01,  9.4611e-01,  1.0118e+00, -1.6861e-01, -6.6695e-01,
         -4.4382e-01, -6.6560e-01,  8.9777e-02, -8.1163e-01,  2.3268e-01,
         -3.6053e-01, -4.1931e-01,  4.5650e-01, -1.6648e-01,  2.8094e-01,
         -4.8273e-01, -7.7533e-01, -3.2518e-01,  7.8259e-02,  1.2031e-01,
         -3.3767e-01,  5.4639e-01, -2.7284e-01, -3.7865e-01,  3.3282e-01,
          9.2420e-01, -5.8171e-02, -9.3855e-01,  4.8351e-01,  2.1065e-01,
         -1.9986e-01, -3.5786e-01,  2.4317e-01, -3.8767e-01, -4.8860e-01,
          3.5934e-01,  3.1637